## Antenas y radios
Duration: 7:00
positive
: **Nota:** Siempre es buena idea revisar que la comunicacion con el dispositivo sige activa al comienzo de una nueva libreta. Aun cuando la conexion a traves de `tcp` deberia seguir activa mientras el servicio siga vigente en el dispositivo, no es necesario volver a establecerla. Sin embargo, en cada nueva libreta **sí** se tiene que importar el modulo `android`.

Vamos a seguir la recomendacion de la nota anterior y verificar que el dispositivo esta conectado a la computadora. 

### Estableciendo conexion con el dispositivo

In [ ]:
!adb forward tcp:9999 tcp:2222

In [ ]:
import android
droide=android.Android()
droide.makeToast("Hola desde Jupyter!")

## Localizacion
En 1995 existen en operacion satelites que proporcionan a los civiles un [Sstema de Posicionamiento Global (en inglés, **GPS**; Global Positioning System)](https://es.wikipedia.org/wiki/GPS) que permite determinar la posicion de cualquier objeto sobre la Tierra.

Los dispositivos moviles modernos tienen una antena receptora de GPS y la API de Python ofrece algunas funciones para extraer valores de ella que permiten conocer la geoposicion actual del dispositivo. Una de ellas es

### <sub>getLastKnownLocation</sub> ###
```
getLastKnownLocation()

Returns the last known location of the device.

Returns:
  A map of location information by provider.
```

In [ ]:
droide.getLastKnownLocation()

## Parentesis
Para desarrollar aplicaciones relacionadas a informacion de geoposicion es util contar con una herramienta para visualizar mapas. La libreria `matplotlib` cuenta con funcionalidad elemental para graficos simples, por ejemplo graficas de dispersion, barras, etc. Por supuesto los pares de latitud,longitud pueden visualizarse con funnciones disponibles en esta libreria sin embargo, no muestran capas de referencia a la geolocalizacion (mapas). Aunque existen varias librearias que proporcionan funciones utiles y flexibles para visualizar mapas, entre ellas `basemaps`, `ipyleaflet`, `bokeh`, `gmaps`, etc. vamos a utilizar nuestro dispositivo que ya contiene un a "libreria" de mapas instalada: la aplicacion de mapas de Google.

### Abrir una aplicacion en el dispositivo
Primero veamos como abrir una aplicacion en el dispositivo desde Python. Podemos ver la lista de aplicaciones en el dispositivo con la siguiente instruccion:

In [ ]:
droide.getLaunchableApplications().result

positive
: La lista de aplicaciones varia de acuerdo al modelo y version de sistema que tenga el dispositivo movil.

Podemos ver que la estructura del diccionario es un par de llave:valor de la forma `{u'AppName':u'ClassName'}`. En algunas ocasiones es posible iniciar las aplicaciones usando la instruccion `launch(u'ClassName')`, por ejemplo

In [ ]:
fb = droide.getLaunchableApplications().result['Facebook']
print(fb)

In [ ]:
droide.launch(fb)

Sin embargo no todos las clases que se obtienen del comando `getLaunchableApplications` se ejecutan directamente, por ejemplo no es posible iniciar la aplicacion de mapas si usamos la misma estrategia. Ejecuten el siguiente codigo y observen que no se abre la aplicacion esperada en el dispositivo. 

In [ ]:
mapas = droide.getLaunchableApplications().result['Maps']
print(mapas)
droide.launch(mapas)

La razon es que la clase a la que esta ligada la aplicacion `Maps` no es un programa sino una *actividad*. Para iniciar el programa de mapas podemos usar

### <sub>startActivity</sub> ###
```
startActivity(
 String action,
 String uri[optional],
 String type[optional]: MIME type/subtype of the URI,
 JSONObject extras[optional]: a Map of extras to add to the Intent,
 Boolean wait[optional]: block until the user exits the started activity,
 String packagename[optional]: name of package. If used, requires classname to  
be useful,
 String classname[optional]: name of class. If used, requires packagename to be 
useful)

Starts an activity.
```

como sigue:

In [ ]:
droide.startActivity(
                    'android.intent.action.MAIN',
                    None,
                    None,
                    None,
                    False,
                    'com.google.android.apps.maps',
                    'com.google.android.maps.MapsActivity'
                    )

La aplicacion inicia desplegando el mapa mas reciente. Es posible especificar una localizacion para el mapa si se espcifica el segundo argumneto `uri` de la funcion. Por ejemplo:

In [ ]:
uri = "geo:19.024666568 -97.268665592"
droide.startActivity(
                    'android.intent.action.VIEW',
                    uri,
                    None,
                    None,
                    False,
                    'com.google.android.apps.maps',
                    'com.google.android.maps.MapsActivity'
                    )

Para obtener la mas reciente geolocalizacion del dispositivo podemos usar la siguiente instruccion:

In [ ]:
droide.getLastKnownLocation().result

Observen que la funcion regresa un diccionario con tres llaves: `gps`, `network` y `passive`, cada una con valores que son nuevamente otro diccionario que contienen detalles de la localizacion. Por el momento nos interesan unicamente `latitude` y `longitude`de la informacion proporcionada por el GPS:

In [ ]:
lon,lat=[droide.getLastKnownLocation().result['gps'][l] for l in ('longitude','latitude')]

In [ ]:
uri = "geo:%s %s?z=15"%(lat,lon)
droide.startActivity(
                    'android.intent.action.VIEW',
                    uri,
                    None,
                    None,
                    False,
                    'com.google.android.apps.maps',
                    'com.google.android.maps.MapsActivity'
                    )

### Invocando aplicaciones
la funcion `startActivity` puede usarse para iniciar aplicaciones usando las opciones de otras formas:

In [ ]:
droide.startActivity('android.intent.action.VIEW', 
      'vnd.youtube:1_u0NmrFM0c', None, None, False, None, None)

### Ejercicio
Busca en internet las coordenadas de alguna localizacion de interes para ti (puedes encontrar idean [en esta pagina](https://latitude.to/articles-by-country/mx/mexico/)) y muestra en el mapa la posicion desde 

## Navegacion
En los ejemplos previos usamos la geolocalizacion mas reciente que "recuerda" el dispositivo. Podemos utilizar la informacion de GPS para conocer nuestra ubicacion actual y para darle seguimiento a cambios en la geolocalizacion.

positive
: El uso continuo del GPS puede reducir la bateria considerablemente. En el siguiente laboratorio vamos a monitorear la bateria y podran darse cuenta de esto con los datos de su dispositivo. 

Para conocer la posicion actual del dispositivo podemos usar

### <sub>startLocating</sub> ###
```
startLocating(
 Integer minDistance[optional, default 60000]: minimum time between updates in  
milliseconds,
 Integer minUpdateDistance[optional, default 30]: minimum distance between      
updates in meters)

Starts collecting location data.

Generates "location" events.
```

In [ ]:
!adb forward tcp:9999 tcp:2222
import android
droide=android.Android()
droide.makeToast("Hola desde Jupyter!")

In [ ]:
droide.startLocating()

In [ ]:
print "reading GPS ..." 
event = droide.eventWaitFor('location',10000).result

In [ ]:
print event

In [ ]:
if event['name'] == "location": 
    try: 
        lat = str(event['data']['gps']['latitude']) 
        lng = str(event['data']['gps']['longitude']) 
    except KeyError: 
        lat = str(event['data']['fused']['latitude']) 
        lng = str(event['data']['fused']['longitude'])
    latlng = 'lat: ' + lat + ' lng: ' + lng 
    print droide.makeToast("Coordenadas: %s"%latlng)

In [ ]:
droide.stopLocating()

In [ ]:
droide.startLocating()

event = droide.eventWaitFor('location',10000).result

if event['name'] == "location": 
    try: 
        lat = str(event['data']['gps']['latitude']) 
        lng = str(event['data']['gps']['longitude']) 
    except KeyError: 
        lat = str(event['data']['fused']['latitude']) 
        lng = str(event['data']['fused']['longitude'])
    latlng = 'lat: ' + lat + '\nlng: ' + lng 
    print droide.makeToast("Coordenadas: \n %s"%latlng)

droide.stopLocating()

In [ ]:
droide.geocode(lat, lon).result

In [ ]:
uri = "google.navigation:q=Los Pinos CDMX"
droide.startActivity(
                    'android.intent.action.VIEW',
                    uri,
                    None,
                    None,
                    False,
                    'com.google.android.apps.maps',
                    'com.google.android.maps.MapsActivity'
                    )

## Enlaces y recursos externos

### Android
* https://stackoverflow.com/questions/48502915/get-list-of-all-apps-with-qpython-sl4a
* https://stackoverflow.com/questions/2662531/launching-google-maps-directions-via-an-intent-on-android
* https://stackoverflow.com/questions/48978716/open-html-file-on-android-with-python-androidhelper-sl4a
* https://stackoverflow.com/questions/6205827/how-to-open-standard-google-map-application-from-my-application
* https://developer.android.com/guide/appendix/g-app-intents.html
* https://android.stackexchange.com/questions/119144/android-tethering-shell-to-python-code-sl4a-api
 
### SL4A
* https://www.pythoncentral.io/series/python-sl4a-android-scripting-layer-tutorial/